# 微笑模拟调用样例

In [ ]:
# 加载所需要的包
import os
import cv2
import glob
import base64
import time
import requests
import json
import urllib
import numpy as np
import matplotlib.pyplot as plt

<h2>定义调用规则</h2>
请根据您从我方获取的信息修改以下代码块

In [ ]:
# 朝厚服务请求地址，随api文档发送
base_url = "<服务请求地址>"

# 朝厚文件服务地址，随api文档发送
file_server_url = "<服务文件服务器地址>"

# 必须传入鉴权 Header。请保护好TOKEN!!! 如果泄露请立即联系我们重置，所有使用该TOKEN的任务都会向您的账户计费
zh_token = "<贵司服务Token, 随合同发送>" # 调用所有的API都必须传入token用作鉴权

user_group = "APIClient" # 用户组，一般为 APIClient

# 贵司user_id, 随api文档发送
user_id = "<贵司user_id>"

# 如果您收到了creds.json, 下面将直接读取
if os.path.exists('../../creds.json'):
    creds = json.load(open('../../creds.json', 'r'))
    base_url = creds['base_url']
    file_server_url = creds['file_server_url']
    zh_token = creds['zh_token']
    user_id = creds['user_id']
    print("loaded creds from creds.json")

In [ ]:
def upload_file(file_name):
    ext = file_name.split('.')[-1]
    data = open('../../data/' + file_name, 'rb').read()
    resp = requests.get(file_server_url + f"/scratch/{user_group}/{user_id}/upload_url?" +
                        f"postfix={ext}", # 必须指定 postfix, 即文件后缀名
                        headers={"X-ZH-TOKEN": zh_token}) # 获取带签名的上传地址
    resp.raise_for_status()

    upload_url = resp.text[1:-1] # 返回为一个单字符串JSON "string", 这里也可以用json.loads(resp.text)

    resp = requests.put(upload_url, data) # 上传至云储存服务不需要带鉴权头

    resp.raise_for_status()
    path = "/".join(urllib.parse.urlparse(upload_url).path.lstrip("/").split("/")[3:])
    urn = f"urn:zhfile:o:s:{user_group}:{user_id}:{path}"
    return urn

def run_job_and_get_results(json_call, timeout_sec):
    headers = {
      "Content-Type": "application/json",
      "X-ZH-TOKEN": zh_token
    }

    url = base_url + '/run'

    response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
    response.raise_for_status()
    create_result = response.json()
    run_id = create_result['run_id']
    print("workflow id is", run_id)
    url = base_url + f"/run/{run_id}"

    start_time = time.time()
    while time.time()-start_time < timeout_sec:
        time.sleep(0.3)
        response = requests.request("GET", url, headers=headers)
        result = response.json()
        if result['completed'] or result['failed']:
            break

    if not result['completed']:
        if result['failed']:
            raise ValueError("API failed due to " + str(result['reason_public']))
        raise TimeoutError("API timeout")

    print("API finished in {}s".format(time.time()-start_time))
    url = base_url + f"/data/{run_id}"
    response = requests.request("GET", url, headers=headers)
    return response.json()

def show_img(urn):
    imbytes = requests.get(file_server_url + f"/file/download?" + urllib.parse.urlencode({
                        "urn": urn}),
                        headers={"X-ZH-TOKEN": zh_token}).content
    imarray = np.asarray(bytearray(imbytes), dtype=np.uint8)
    plt.imshow(cv2.imdecode(imarray, cv2.IMREAD_COLOR)[...,::-1])

## 矫正后微笑模拟

以下代码输出矫正后微笑模拟，具体可参考 https://www.chohotech.com/docs/cloud-zh/#/module/smile-sim-lip-preserve-1

In [ ]:
json_call = {
  "spec_group": "smile",
  "spec_name": "smile-sim-lip-preserve",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {"image": upload_file("face_smile.jpg")}
}
result = run_job_and_get_results(json_call, 10)

In [ ]:
show_img(result['result']['image'])

## 固定矫正模拟

以下代码输出佩戴固定矫治器的模拟，具体可参考 https://www.chohotech.com/docs/cloud-zh/#/module/archwire-sim-1

In [ ]:
json_call = {
  "spec_group": "smile",
  "spec_name": "archwire-sim",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {"image": upload_file("face_smile.jpg")}
}
result = run_job_and_get_results(json_call, 10)

In [ ]:
show_img(result['result']['image'])

## 隐形矫正模拟

以下代码输出佩戴隐形牙套的模拟，具体可参考 https://www.chohotech.com/docs/cloud-zh/#/module/aligner-sim-1

In [ ]:
json_call = {
  "spec_group": "smile",
  "spec_name": "aligner-sim",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {"image": upload_file("face_smile.jpg")}
}
result = run_job_and_get_results(json_call, 10)

In [ ]:
show_img(result['result']['image'])